In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
data = "https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/d12a1a3cdb62c5646282ebe51a4a1e1cf93f47a6/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv"


In [3]:
!wget $data

--2023-10-09 16:56:38--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/d12a1a3cdb62c5646282ebe51a4a1e1cf93f47a6/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving proxy-web.micron.com (proxy-web.micron.com)... 185.46.212.88
Connecting to proxy-web.micron.com (proxy-web.micron.com)|185.46.212.88|:80... connected.
Proxy request sent, awaiting response... 200 OK
Length: 977501 (955K) [text/plain]
Saving to: ‘WA_Fn-UseC_-Telco-Customer-Churn.csv’

WA_Fn-UseC_-Telco-C 100%[===================>] 954.59K  1.99MB/s    in 0.5s    

2023-10-09 16:56:39 (1.99 MB/s) - ‘WA_Fn-UseC_-Telco-Customer-Churn.csv’ saved [977501/977501]



In [5]:
df = pd.read_csv('data-week-3.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [7]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)